<a href="https://colab.research.google.com/github/Ahmed11Raza/Python-Projects/blob/main/Personal_Library_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import json
import datetime
from typing import List, Dict, Optional, Union


class Book:
    def __init__(
        self,
        title: str,
        author: str,
        isbn: str = "",
        genre: str = "",
        publication_year: Optional[int] = None,
        publisher: str = "",
        pages: Optional[int] = None,
        status: str = "Unread",
        date_added: Optional[str] = None,
        rating: Optional[float] = None,
        notes: str = "",
    ):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.genre = genre
        self.publication_year = publication_year
        self.publisher = publisher
        self.pages = pages
        self.status = status  # Unread, Reading, Completed
        self.date_added = date_added or datetime.datetime.now().strftime("%Y-%m-%d")
        self.rating = rating  # 1-5 scale
        self.notes = notes

    def to_dict(self) -> Dict:
        return {
            "title": self.title,
            "author": self.author,
            "isbn": self.isbn,
            "genre": self.genre,
            "publication_year": self.publication_year,
            "publisher": self.publisher,
            "pages": self.pages,
            "status": self.status,
            "date_added": self.date_added,
            "rating": self.rating,
            "notes": self.notes,
        }

    @classmethod
    def from_dict(cls, data: Dict) -> "Book":
        return cls(**data)

    def __str__(self) -> str:
        rating_str = f"{self.rating}/5" if self.rating is not None else "Not rated"
        return f"{self.title} by {self.author} ({self.publication_year}) - {self.status} - {rating_str}"


class LibraryManager:
    def __init__(self, data_file: str = "library.json"):
        self.data_file = data_file
        self.books: List[Book] = []
        self.load_library()

    def load_library(self) -> None:
        """Load the library from the JSON file."""
        if os.path.exists(self.data_file):
            try:
                with open(self.data_file, "r") as file:
                    books_data = json.load(file)
                    self.books = [Book.from_dict(book) for book in books_data]
                print(f"Loaded {len(self.books)} books from {self.data_file}")
            except json.JSONDecodeError:
                print(f"Error decoding {self.data_file}. Starting with an empty library.")
        else:
            print(f"No library file found at {self.data_file}. Starting with an empty library.")

    def save_library(self) -> None:
        """Save the library to the JSON file."""
        with open(self.data_file, "w") as file:
            json.dump([book.to_dict() for book in self.books], file, indent=2)
        print(f"Library saved to {self.data_file}")

    def add_book(self, book: Book) -> None:
        """Add a book to the library."""
        self.books.append(book)
        print(f'Added "{book.title}" by {book.author} to the library.')
        self.save_library()

    def remove_book(self, book_title: str) -> bool:
        """Remove a book from the library by title."""
        initial_count = len(self.books)
        self.books = [book for book in self.books if book.title.lower() != book_title.lower()]

        if len(self.books) < initial_count:
            print(f'Removed "{book_title}" from the library.')
            self.save_library()
            return True
        else:
            print(f'Book "{book_title}" not found in the library.')
            return False

    def get_book_by_title(self, title: str) -> Optional[Book]:
        """Find a book by its title."""
        for book in self.books:
            if book.title.lower() == title.lower():
                return book
        return None

    def update_book_status(self, title: str, status: str) -> bool:
        """Update the reading status of a book."""
        book = self.get_book_by_title(title)
        if book:
            book.status = status
            print(f'Updated "{book.title}" status to {status}.')
            self.save_library()
            return True
        else:
            print(f'Book "{title}" not found in the library.')
            return False

    def rate_book(self, title: str, rating: float) -> bool:
        """Add a rating to a book (1-5 scale)."""
        if not 0 <= rating <= 5:
            print("Rating must be between 0 and 5.")
            return False

        book = self.get_book_by_title(title)
        if book:
            book.rating = rating
            print(f'Rated "{book.title}" as {rating}/5.')
            self.save_library()
            return True
        else:
            print(f'Book "{title}" not found in the library.')
            return False

    def add_notes(self, title: str, notes: str) -> bool:
        """Add notes to a book."""
        book = self.get_book_by_title(title)
        if book:
            book.notes = notes
            print(f'Added notes to "{book.title}".')
            self.save_library()
            return True
        else:
            print(f'Book "{title}" not found in the library.')
            return False

    def search_books(self, query: str) -> List[Book]:
        """Search for books by title, author, or genre."""
        query = query.lower()
        results = []

        for book in self.books:
            if (query in book.title.lower() or
                query in book.author.lower() or
                query in book.genre.lower()):
                results.append(book)

        return results

    def list_books(self, filter_by: Optional[str] = None, value: Optional[str] = None) -> List[Book]:
        """List all books, optionally filtered by a field and value."""
        if not filter_by:
            return self.books

        filtered_books = []
        for book in self.books:
            if hasattr(book, filter_by):
                book_value = getattr(book, filter_by)
                if isinstance(book_value, str) and book_value.lower() == value.lower():
                    filtered_books.append(book)

        return filtered_books

    def get_statistics(self) -> Dict[str, Union[int, Dict[str, int]]]:
        """Get statistics about the library."""
        total_books = len(self.books)
        genres = {}
        statuses = {"Unread": 0, "Reading": 0, "Completed": 0}
        authors = {}

        for book in self.books:
            # Count genres
            if book.genre:
                genres[book.genre] = genres.get(book.genre, 0) + 1

            # Count statuses
            if book.status:
                statuses[book.status] = statuses.get(book.status, 0) + 1

            # Count authors
            if book.author:
                authors[book.author] = authors.get(book.author, 0) + 1

        return {
            "total_books": total_books,
            "genres": genres,
            "statuses": statuses,
            "authors": authors
        }


def create_sample_books() -> List[Book]:
    """Create some sample books for testing."""
    return [
        Book(
            title="To Kill a Mockingbird",
            author="Harper Lee",
            isbn="9780061120084",
            genre="Fiction",
            publication_year=1960,
            publisher="HarperCollins",
            pages=281,
            status="Completed",
            rating=5.0,
            notes="Classic novel about racial injustice in the American South.",
        ),
        Book(
            title="1984",
            author="George Orwell",
            isbn="9780451524935",
            genre="Dystopian",
            publication_year=1949,
            publisher="Signet Classics",
            pages=328,
            status="Reading",
            rating=4.5,
        ),
        Book(
            title="The Great Gatsby",
            author="F. Scott Fitzgerald",
            isbn="9780743273565",
            genre="Fiction",
            publication_year=1925,
            publisher="Scribner",
            pages=180,
            status="Unread",
        ),
    ]


def main_cli():
    """Run a command-line interface for the library manager."""
    library = LibraryManager()

    # Add sample books if the library is empty
    if not library.books:
        print("Adding sample books to the empty library...")
        for book in create_sample_books():
            library.add_book(book)

    while True:
        print("\n==== Personal Library Manager ====")
        print("1. View All Books")
        print("2. Add a Book")
        print("3. Remove a Book")
        print("4. Update Book Status")
        print("5. Rate a Book")
        print("6. Add Notes to a Book")
        print("7. Search Books")
        print("8. View Library Statistics")
        print("9. Exit")

        choice = input("\nEnter your choice (1-9): ")

        if choice == "1":
            print("\n--- Your Books ---")
            for i, book in enumerate(library.books, 1):
                print(f"{i}. {book}")

        elif choice == "2":
            title = input("Enter book title: ")
            author = input("Enter author: ")

            isbn = input("Enter ISBN (optional): ")
            genre = input("Enter genre (optional): ")

            pub_year = input("Enter publication year (optional): ")
            publication_year = int(pub_year) if pub_year.isdigit() else None

            publisher = input("Enter publisher (optional): ")

            pages_str = input("Enter number of pages (optional): ")
            pages = int(pages_str) if pages_str.isdigit() else None

            status = input("Enter status (Unread, Reading, Completed) [default: Unread]: ")
            if status not in ["Unread", "Reading", "Completed"]:
                status = "Unread"

            new_book = Book(
                title=title,
                author=author,
                isbn=isbn,
                genre=genre,
                publication_year=publication_year,
                publisher=publisher,
                pages=pages,
                status=status
            )

            library.add_book(new_book)

        elif choice == "3":
            title = input("Enter the title of the book to remove: ")
            library.remove_book(title)

        elif choice == "4":
            title = input("Enter book title: ")
            print("Statuses: Unread, Reading, Completed")
            status = input("Enter new status: ")
            if status in ["Unread", "Reading", "Completed"]:
                library.update_book_status(title, status)
            else:
                print("Invalid status. Use Unread, Reading, or Completed.")

        elif choice == "5":
            title = input("Enter book title: ")
            try:
                rating = float(input("Enter rating (0-5): "))
                library.rate_book(title, rating)
            except ValueError:
                print("Please enter a valid number for rating.")

        elif choice == "6":
            title = input("Enter book title: ")
            notes = input("Enter your notes: ")
            library.add_notes(title, notes)

        elif choice == "7":
            query = input("Enter search term: ")
            results = library.search_books(query)

            if results:
                print(f"\nFound {len(results)} matching books:")
                for i, book in enumerate(results, 1):
                    print(f"{i}. {book}")
            else:
                print("No books found matching your query.")

        elif choice == "8":
            stats = library.get_statistics()
            print("\n--- Library Statistics ---")
            print(f"Total books: {stats['total_books']}")

            print("\nBooks by status:")
            for status, count in stats['statuses'].items():
                print(f"  {status}: {count}")

            print("\nTop genres:")
            for genre, count in sorted(stats['genres'].items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"  {genre}: {count}")

            print("\nTop authors:")
            for author, count in sorted(stats['authors'].items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"  {author}: {count} books")

        elif choice == "9":
            print("Exiting. Your library has been saved.")
            break

        else:
            print("Invalid choice. Please enter a number between 1 and 9.")


if __name__ == "__main__":
    main_cli()

Loaded 4 books from library.json

==== Personal Library Manager ====
1. View All Books
2. Add a Book
3. Remove a Book
4. Update Book Status
5. Rate a Book
6. Add Notes to a Book
7. Search Books
8. View Library Statistics
9. Exit

Enter your choice (1-9): 9
Exiting. Your library has been saved.


SyntaxError: incomplete input (<ipython-input-2-713f811d9eb5>, line 402)